In [326]:
#Importing dependencies
import requests
import json
import requests
import wikipedia as wp
from pprint import pprint
import pandas as pd
import gmaps
import matplotlib.pyplot as plt
from config import g_key
import numpy as np

In [322]:
#Creating a function to pull in hospital data

def hospital_data():
    html = wp.page("List_of_hospitals_in_Georgia_(U.S._state)").html().encode("UTF-8")
    try: 
        df = pd.read_html(html)[1]  # Try 2nd table first as most pages contain contents table first
    except IndexError:
        df = pd.read_html(html)[0]
    return df


hospital_df= hospital_data()

In [5]:
#Get Place ID of each hospital (https://developers.google.com/places/web-service/search)
hospital_df["Place ID"] = ''

params = {"keyword" : "Hospital",
          "key" : g_key,
          "inputtype" : "textquery"
         }


for index, row in hospital_df.iterrows():
    try:
        url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
        params["input"] = row["Hospital name"]
        response = requests.get(url, params=params).json()
        
        #Get Place ID
        place_id = response["candidates"][0]["place_id"]
        hospital_df.loc[index,"Place ID"] = place_id
    except:
        pass

        

In [323]:
hospital_df.head()

,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes
0,AdventHealth Gordon,Calhoun,Gordon,69,NaN,NaN,NaN,AdventHealth,Formerly Gordon Hospital
1,AdventHealth Murray,Chatsworth,Murray,29,NaN,NaN,NaN,AdventHealth,Formerly Murray Medical Center
2,Archbold Memorial Hospital,Thomasville,Thomas,264,Level II,NaN,1925,Archbold,NaN
3,Archbold - Brooks County Hospital,Quitman,Brooks,25,NaN,NaN,NaN,Archbold,NaN
4,Archbold - Grady General Hospital,Cairo,Grady,60,NaN,NaN,1960,Archbold,NaN


In [7]:
#Use place_id (https://developers.google.com/places/web-service/details) to pull in remaining address information
hospital_df["Latitude"] = ''
hospital_df["Longtitude"] = ''
hospital_df["Street Address"] = ''
hospital_df["City"] = ''
hospital_df["County"] = ''
hospital_df["State"] = ''
hospital_df["Zip Code"] = ''
hospital_df["Full Address"] = ''


params = {"key" : g_key
         }


for index, row in hospital_df.iterrows():
    try:
        url = "https://maps.googleapis.com/maps/api/place/details/json"
        params["place_id"] = row["Place ID"]
        response = requests.get(url, params=params).json()
        
        #Pulling new values
        lat = response['result']['geometry']['location']['lat']
        long = response['result']['geometry']['location']['lng']
        street_number = response['result']['address_components'][0]['long_name']
        street_name = response['result']['address_components'][1]['long_name']
        address = f"{street_number} {street_name}"

        city = response['result']['address_components'][2]['long_name']
        county = response['result']['address_components'][3]['long_name']
        state = response['result']['address_components'][4]['long_name']
        zip_code = response['result']['address_components'][6]['long_name']
        full_address = response['result']['formatted_address']
        
        #Putting new values in Hospital DF
        hospital_df.loc[index,"Latitude"] = lat
        hospital_df.loc[index,"Longtitude"] = long
        hospital_df.loc[index,"Street Address"] = address
        hospital_df.loc[index,"City"] = city
        hospital_df.loc[index,"County"] = county
        hospital_df.loc[index,"State"] = state
        hospital_df.loc[index,"Zip Code"] = zip_code
        hospital_df.loc[index,"Full Address"] = full_address
        
    except:
        pass

In [8]:
#Printing results
hospital_df

,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,NaN,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.5102,-84.9263,1035 Red Bud Road Northeast,Georgia,30701,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,NaN,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.78,-84.783,707 Old Dalton Ellijay Road,Georgia,30705,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,Level II,NaN,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.8252,-83.9721,915 Gordon Avenue,Georgia,31792,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,NaN,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795,-83.5591,903 North Court Street,Georgia,31643,"903 N Court St, Quitman, GA 31643, USA"
4,Archbold - Grady General Hospital,Cairo,Grady County,60,NaN,NaN,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.8638,-84.1991,1155 5th Street Southeast,Georgia,39828,"1155 5th St SE, Cairo, GA 39828, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,WellStar Paulding Hospital,Hiram,Paulding County,216,NaN,NaN,1954,WellStar,NaN,ChIJMzJn-How9YgRhhfE1kXnu30,33.9011,-84.7816,2518 Jimmy Lee Smith Parkway,Georgia,30141,"2518 Jimmy Lee Smith Pkwy, Hiram, GA 30141, USA"
133,WellStar Spalding Regional Hospital,Griffin,Spalding County,160,NaN,NaN,NaN,WellStar,Formerly Spalding Regional Medical Center,ChIJyexMgeBh9IgRxUd1HX227ns,33.2393,-84.2673,601 South 8th Street,Georgia,30224,"601 S 8th St, Griffin, GA 30224, USA"
134,WellStar Sylvan Grove Hospital,Jackson,Butts County,21,NaN,NaN,NaN,WellStar,NaN,ChIJB5o2d9JB9IgRr-4NjnGt52c,33.3038,-83.9778,1050 McDonough Road,Georgia,30233,"1050 McDonough Rd, Jackson, GA 30233, USA"
135,WellStar West Georgia Medical Center,LaGrange,Troup County,542,Level III,NaN,1937,WellStar,NaN,ChIJ40ZJ5vtWi4gRK6PnDb-BAPU,33.0309,-85.0586,1514 Vernon Road,Georgia,30240,"1514 Vernon Rd, LaGrange, GA 30240, USA"


In [9]:
#Set aside hospitals with correct data
correct_hospitals = hospital_df.loc[hospital_df["State"] == "Georgia"]
correct_hospitals

,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,NaN,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.5102,-84.9263,1035 Red Bud Road Northeast,Georgia,30701,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,NaN,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.78,-84.783,707 Old Dalton Ellijay Road,Georgia,30705,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,Level II,NaN,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.8252,-83.9721,915 Gordon Avenue,Georgia,31792,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,NaN,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795,-83.5591,903 North Court Street,Georgia,31643,"903 N Court St, Quitman, GA 31643, USA"
4,Archbold - Grady General Hospital,Cairo,Grady County,60,NaN,NaN,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.8638,-84.1991,1155 5th Street Southeast,Georgia,39828,"1155 5th St SE, Cairo, GA 39828, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,WellStar Paulding Hospital,Hiram,Paulding County,216,NaN,NaN,1954,WellStar,NaN,ChIJMzJn-How9YgRhhfE1kXnu30,33.9011,-84.7816,2518 Jimmy Lee Smith Parkway,Georgia,30141,"2518 Jimmy Lee Smith Pkwy, Hiram, GA 30141, USA"
133,WellStar Spalding Regional Hospital,Griffin,Spalding County,160,NaN,NaN,NaN,WellStar,Formerly Spalding Regional Medical Center,ChIJyexMgeBh9IgRxUd1HX227ns,33.2393,-84.2673,601 South 8th Street,Georgia,30224,"601 S 8th St, Griffin, GA 30224, USA"
134,WellStar Sylvan Grove Hospital,Jackson,Butts County,21,NaN,NaN,NaN,WellStar,NaN,ChIJB5o2d9JB9IgRr-4NjnGt52c,33.3038,-83.9778,1050 McDonough Road,Georgia,30233,"1050 McDonough Rd, Jackson, GA 30233, USA"
135,WellStar West Georgia Medical Center,LaGrange,Troup County,542,Level III,NaN,1937,WellStar,NaN,ChIJ40ZJ5vtWi4gRK6PnDb-BAPU,33.0309,-85.0586,1514 Vernon Road,Georgia,30240,"1514 Vernon Rd, LaGrange, GA 30240, USA"


In [10]:
#strip out hospitals with shifted/bad data in order to clean it up
fix_hospitals = hospital_df.loc[hospital_df["State"] != "Georgia"]
fix_hospitals

,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
6,Augusta University Children's Hospital of Georgia,Medical District,Augusta,154,NaN,No,NaN,Augusta University,NaN,ChIJoyV_F0jM-YgRqAe60-Abauk,33.4711,-81.9878,1446 Harper Street,Richmond County,United States,"1446 Harper St, Augusta, GA 30912, USA"
7,Augusta University Medical Center,Medical District,Augusta,478,Level I,Yes,NaN,Augusta University,NaN,ChIJyX-W70nM-YgRtaRCmY78n_g,33.4713,-81.9879,1120 15th Street,Richmond County,United States,"1120 15th St, Augusta, GA 30912, USA"
12,Candler Hospital,Midtown,Savannah,384,NaN,NaN,1804,SJCHS,"Oldest hospital in Georgia, and second oldest ...",ChIJpb2fD5N1-4gRM-PXE90FV6c,32.0278,-81.1001,5353 Reynolds Street,Chatham County,United States,"5353 Reynolds St, Savannah, GA 31405, USA"
15,CHI Memorial Hospital Georgia,Circle,Fort Oglethorpe,36,NaN,NaN,NaN,CHI,"Formerly Cornerstone Medical Center, previousl...",ChIJoSI7iAdoYIgRYFrfQLt3nOA,34.9469,-85.264,100 Gross Crescent,Catoosa County,United States,"100 Gross Crescent, Fort Oglethorpe, GA 30742,..."
17,Children's Healthcare of Atlanta Hughes Spalding,Downtown Atlanta,Atlanta,NaN,NaN,Yes,1952,CHOA,NaN,ChIJE0RlP4kD9YgRf-FdJvSfWXw,33.7539,-84.382,35 Jesse Hill Junior Drive Southeast,Fulton County,United States,"35 Jesse Hill Jr Dr SE, Atlanta, GA 30303, USA"
18,Children's Healthcare of Atlanta Scottish Rite,Perimeter Center,Atlanta,NaN,Level II Peds,Yes,1915,CHOA,NaN,ChIJV-6vqjQJ9YgRhvQE1FyLn-g,33.9062,-84.3532,1001 Johnson Ferry Road Northeast,Fulton County,United States,"1001 Johnson Ferry Rd NE, Atlanta, GA 30342, USA"
30,Eastside Medical Center,Peachtree Parkway,Peachtree Corners,249,NaN,NaN,NaN,HCA,NaN,ChIJaTUBHwOh9YgRoHi7L2zoz4E,33.964,-84.2259,100 5707,Gwinnett County,United States,"5707 Peachtree Pkwy #100, Peachtree Corners, G..."
37,Emory Saint Joseph's Hospital,Perimeter Center,Atlanta,276,NaN,NaN,1880,Emory,Formerly Saint Joseph's Hospital of Atlanta,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.9084,-84.3499,5665 Peachtree Dunwoody Road,Fulton County,United States,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,..."
39,Emory University Hospital Midtown,SoNo,Atlanta,511,NaN,Yes,1908,Emory,Formerly Crawford W. Long Memorial Hospital,ChIJ42AWonoE9YgRonLLiSdYxtA,33.7687,-84.3864,550 Peachtree Street Northeast,Fulton County,United States,"550 Peachtree St NE, Atlanta, GA 30308, USA"
43,Fannin Regional Hospital,Old Highway 5,Blue Ridge,50,NaN,NaN,NaN,â,NaN,ChIJN7SfeV2vX4gRu5N9mgngcXo,34.9177,-84.3743,North 2855,Fannin County,United States,"2855 Old Hwy 5 North, Blue Ridge, GA 30513, USA"


In [11]:
#Correct the bad address data for bad hospitals by shifting indices in the for loop
fix_hospitals["Latitude"] = ''
fix_hospitals["Longtitude"] = ''
fix_hospitals["Street Address"] = ''
fix_hospitals["City"] = ''
fix_hospitals["County"] = ''
fix_hospitals["State"] = ''
fix_hospitals["Zip Code"] = ''
fix_hospitals["Full Address"] = ''


params = {"key" : g_key
         }


for index, row in fix_hospitals.iterrows():
    try:
        url = "https://maps.googleapis.com/maps/api/place/details/json"
        params["place_id"] = row["Place ID"]
        response = requests.get(url, params=params).json()
        lat = response['result']['geometry']['location']['lat']
        long = response['result']['geometry']['location']['lng']
        street_number = response['result']['address_components'][1]['long_name']
        street_name = response['result']['address_components'][2]['long_name']
        address = f"{street_number} {street_name}"


        city = response['result']['address_components'][3]['long_name']
        county = response['result']['address_components'][4]['long_name']
        state = response['result']['address_components'][5]['long_name']
        zip_code = response['result']['address_components'][7]['long_name']
        full_address = response['result']['formatted_address']
        
        #Putting new values in Hospital DF
        fix_hospitals.loc[index,"Latitude"] = lat
        fix_hospitals.loc[index,"Longtitude"] = long
        fix_hospitals.loc[index,"Street Address"] = address
        fix_hospitals.loc[index,"City"] = city
        fix_hospitals.loc[index,"County"] = county
        fix_hospitals.loc[index,"State"] = state
        fix_hospitals.loc[index,"Zip Code"] = zip_code
        fix_hospitals.loc[index,"Full Address"] = full_address
        
    except:
        pass

C:\Users\kledesma\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\kledesma\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kledesma\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [12]:
#Getting a look at the formatting
params = {"key" : g_key
         }


url = "https://maps.googleapis.com/maps/api/place/details/json"
params["place_id"] = 'ChIJHSV3CO7aWIgRzsq072J_i_c'
response = requests.get(url, params=params).json()


pprint(response)

{'html_attributions': [],
 'result': {'address_components': [{'long_name': 'Clayton',
                                    'short_name': 'Clayton',
                                    'types': ['locality', 'political']},
                                   {'long_name': 'Rabun County',
                                    'short_name': 'Rabun County',
                                    'types': ['administrative_area_level_2',
                                              'political']},
                                   {'long_name': 'Georgia',
                                    'short_name': 'GA',
                                    'types': ['administrative_area_level_1',
                                              'political']},
                                   {'long_name': 'United States',
                                    'short_name': 'US',
                                    'types': ['country', 'political']},
                                   {'long_name': '30525',
     

In [13]:
#Verifying data is fixed for fix_hospitals df
fix_hospitals

,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
6,Augusta University Children's Hospital of Georgia,Augusta,Richmond County,154,NaN,No,NaN,Augusta University,NaN,ChIJoyV_F0jM-YgRqAe60-Abauk,33.4711,-81.9878,Harper Street Medical District,Georgia,30912,"1446 Harper St, Augusta, GA 30912, USA"
7,Augusta University Medical Center,Augusta,Richmond County,478,Level I,Yes,NaN,Augusta University,NaN,ChIJyX-W70nM-YgRtaRCmY78n_g,33.4713,-81.9879,15th Street Medical District,Georgia,30912,"1120 15th St, Augusta, GA 30912, USA"
12,Candler Hospital,Savannah,Chatham County,384,NaN,NaN,1804,SJCHS,"Oldest hospital in Georgia, and second oldest ...",ChIJpb2fD5N1-4gRM-PXE90FV6c,32.0278,-81.1001,Reynolds Street Midtown,Georgia,31405,"5353 Reynolds St, Savannah, GA 31405, USA"
15,CHI Memorial Hospital Georgia,Fort Oglethorpe,Catoosa County,36,NaN,NaN,NaN,CHI,"Formerly Cornerstone Medical Center, previousl...",ChIJoSI7iAdoYIgRYFrfQLt3nOA,34.9469,-85.264,Gross Crescent Circle,Georgia,30742,"100 Gross Crescent, Fort Oglethorpe, GA 30742,..."
17,Children's Healthcare of Atlanta Hughes Spalding,Atlanta,Fulton County,NaN,NaN,Yes,1952,CHOA,NaN,ChIJE0RlP4kD9YgRf-FdJvSfWXw,33.7539,-84.382,Jesse Hill Junior Drive Southeast Downtown Atl...,Georgia,30303,"35 Jesse Hill Jr Dr SE, Atlanta, GA 30303, USA"
18,Children's Healthcare of Atlanta Scottish Rite,Atlanta,Fulton County,NaN,Level II Peds,Yes,1915,CHOA,NaN,ChIJV-6vqjQJ9YgRhvQE1FyLn-g,33.9062,-84.3532,Johnson Ferry Road Northeast Perimeter Center,Georgia,30342,"1001 Johnson Ferry Rd NE, Atlanta, GA 30342, USA"
30,Eastside Medical Center,Peachtree Corners,Gwinnett County,249,NaN,NaN,NaN,HCA,NaN,ChIJaTUBHwOh9YgRoHi7L2zoz4E,33.964,-84.2259,5707 Peachtree Parkway,Georgia,30092,"5707 Peachtree Pkwy #100, Peachtree Corners, G..."
37,Emory Saint Joseph's Hospital,Atlanta,Fulton County,276,NaN,NaN,1880,Emory,Formerly Saint Joseph's Hospital of Atlanta,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.9084,-84.3499,Peachtree Dunwoody Road Perimeter Center,Georgia,30342,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,..."
39,Emory University Hospital Midtown,Atlanta,Fulton County,511,NaN,Yes,1908,Emory,Formerly Crawford W. Long Memorial Hospital,ChIJ42AWonoE9YgRonLLiSdYxtA,33.7687,-84.3864,Peachtree Street Northeast SoNo,Georgia,30308,"550 Peachtree St NE, Atlanta, GA 30308, USA"
43,Fannin Regional Hospital,Blue Ridge,Fannin County,50,NaN,NaN,NaN,â,NaN,ChIJN7SfeV2vX4gRu5N9mgngcXo,34.9177,-84.3743,2855 Old Highway 5,Georgia,30513,"2855 Old Hwy 5 North, Blue Ridge, GA 30513, USA"


In [14]:
#Concatenating the two DF's back together and printing, doing some checks
cleaned_hospitals = pd.concat([correct_hospitals, fix_hospitals])

cleaned_hospitals["State"].unique()
cleaned_hospitals["City"].unique()
cleaned_hospitals["County"].unique()

array(['Gordon County', 'Murray County', 'Thomas County', 'Brooks County',
       'Grady County', 'Mitchell County', 'Bibb County',
       'Bleckley County', 'Coweta County', 'Candler County',
       'Bartow County', 'Towns County', 'DeKalb County', 'Clinch County',
       'Coffee County', 'Colquitt County', 'Crisp County',
       'Richmond County', 'Dodge County', 'Seminole County',
       'Ben Hill County', 'Bulloch County', 'Effingham County',
       'Elbert County', 'Emanuel County', 'Fulton County', 'Evans County',
       'Laurens County', 'Macon County', 'Floyd County', 'Polk County',
       'Whitfield County', 'Haralson County', 'Houston County',
       'Irwin County', 'Jasper County', 'Jeff Davis County',
       'Jenkins County', 'Liberty County', 'Early County',
       'Toombs County', 'Peach County', 'Decatur County', 'Ware County',
       'Miller County', 'Monroe County', 'Morgan County',
       'Baldwin County', 'Hall County', 'Barrow County', 'Lumpkin County',
       'Jack

In [15]:
#Re-ordering the columns, dropping irrelevant columns
del cleaned_hospitals['Traumacenterdesignation']
del cleaned_hospitals['COTH[2]TeachingHospital']

In [16]:
cleaned_hospitals

,Hospital name,City,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.5102,-84.9263,1035 Red Bud Road Northeast,Georgia,30701,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.78,-84.783,707 Old Dalton Ellijay Road,Georgia,30705,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.8252,-83.9721,915 Gordon Avenue,Georgia,31792,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795,-83.5591,903 North Court Street,Georgia,31643,"903 N Court St, Quitman, GA 31643, USA"
4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.8638,-84.1991,1155 5th Street Southeast,Georgia,39828,"1155 5th St SE, Cairo, GA 39828, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,St. Francis Hospital,Decatur,DeKalb County,360,1950,Emory,NaN,ChIJrzBX-3gH9YgRpeQf21LpXF4,33.7958,-84.277,Dekalb Industrial Way Greater Valley Brook,Georgia,30033,"625 Dekalb Industrial Way, Decatur, GA 30033, USA"
110,St. Joseph's Hospital,Atlanta,Fulton County,330,1875,SJCHS,NaN,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.9084,-84.3499,Peachtree Dunwoody Road Perimeter Center,Georgia,30342,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,..."
111,St. Mary's Hospital,Atlanta,Fulton County,170,NaN,St. Mary's,NaN,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.9084,-84.3499,Peachtree Dunwoody Road Perimeter Center,Georgia,30342,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,..."
126,Wellstar Atlanta Medical Center,Atlanta,Fulton County,460,1901,WellStar,NaN,ChIJH9RgbwsE9YgRCefukbDOojs,33.7629,-84.373,Parkway Drive Northeast Old Fourth Ward,Georgia,30312,"303 Parkway Dr NE, Atlanta, GA 30312, USA"


In [17]:
#write to csv
cleaned_hospitals.to_csv(r"Cleaned_Hospitalsv2.csv")

In [332]:
#Formatting the data
hospital = pd.read_csv("Cleaned_Hospitalsv2.csv")

hospital["Acute Care Beds"].fillna("0", inplace=True)
del hospital['Unnamed: 0']
hospital.head()

,Hospital name,City,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.510205,-84.926304,1035 Red Bud Road Northeast,Georgia,30701.0,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.779956,-84.783038,707 Old Dalton Ellijay Road,Georgia,30705.0,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.825169,-83.972128,915 Gordon Avenue,Georgia,31792.0,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795039,-83.559146,903 North Court Street,Georgia,31643.0,"903 N Court St, Quitman, GA 31643, USA"
4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.863815,-84.199066,1155 5th Street Southeast,Georgia,39828.0,"1155 5th St SE, Cairo, GA 39828, USA"


In [333]:
#isolating more potential bad data
hospital_fix = hospital.loc[(hospital['State'] != 'Georgia')]
hospital_fix

,Hospital name,City,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
126,Jefferson Hospital,Philadelphia,Philadelphia County,37,NaN,â,NaN,ChIJ-W1LT-3HxokRD_nmm7DDoU0,39.949562,-75.158032,South 11th Street Center City East,Pennsylvania,19107.0,"111 S 11th St, Philadelphia, PA 19107, USA"
128,Mountain Lakes Medical Center,NaN,NaN,0,NaN,â,NaN,ChIJHSV3CO7aWIgRzsq072J_i_c,NaN,NaN,NaN,NaN,NaN,NaN
131,Piedmont Henry Hospital,NaN,NaN,277,NaN,Piedmont,NaN,ChIJBduprw5X9IgRygZRl0TyxOs,NaN,NaN,NaN,NaN,NaN,NaN


In [337]:
#Extracting clean data
hospital = hospital.loc[(hospital['State'] == 'Georgia')]
hospital

,Hospital name,City,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.510205,-84.926304,1035 Red Bud Road Northeast,Georgia,30701.0,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.779956,-84.783038,707 Old Dalton Ellijay Road,Georgia,30705.0,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.825169,-83.972128,915 Gordon Avenue,Georgia,31792.0,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795039,-83.559146,903 North Court Street,Georgia,31643.0,"903 N Court St, Quitman, GA 31643, USA"
4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.863815,-84.199066,1155 5th Street Southeast,Georgia,39828.0,"1155 5th St SE, Cairo, GA 39828, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,St. Francis Hospital,Decatur,DeKalb County,360,1950,Emory,NaN,ChIJrzBX-3gH9YgRpeQf21LpXF4,33.795761,-84.276954,Dekalb Industrial Way Greater Valley Brook,Georgia,30033.0,"625 Dekalb Industrial Way, Decatur, GA 30033, USA"
133,St. Joseph's Hospital,Atlanta,Fulton County,330,1875,SJCHS,NaN,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.908422,-84.349851,Peachtree Dunwoody Road Perimeter Center,Georgia,30342.0,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,..."
134,St. Mary's Hospital,Atlanta,Fulton County,170,NaN,St. Mary's,NaN,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.908422,-84.349851,Peachtree Dunwoody Road Perimeter Center,Georgia,30342.0,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,..."
135,Wellstar Atlanta Medical Center,Atlanta,Fulton County,460,1901,WellStar,NaN,ChIJH9RgbwsE9YgRCefukbDOojs,33.762907,-84.372961,Parkway Drive Northeast Old Fourth Ward,Georgia,30312.0,"303 Parkway Dr NE, Atlanta, GA 30312, USA"


In [389]:
#Import Census data
census_data= pd.read_csv("Resources/Clean_Census_Data.csv")

del census_data['Unnamed: 0']
del census_data['State']

In [390]:
#Strip out census data with bad data for median income and population
census_data = census_data.loc[(census_data['Median Income'] > 0)]
census_data

,City,ZIP,Latitude,Longitude,County,Population,Median Income
0,Abbeville,31001,31.972626,-83.33082,Abbeville County,4563.0,20880.0
1,Acworth,30102,34.097998,-84.61830,Cobb County,99683.0,33711.5
2,Adairsville,30103,34.368647,-84.92452,Bartow County,16138.0,24570.0
3,Adel,31620,31.133004,-83.42736,Dallas County,10836.0,25632.0
4,Adrian,31002,32.549854,-82.56948,Lenawee County,3071.0,19562.0
...,...,...,...,...,...,...,...
509,Wrens,30833,33.200325,-82.37625,Jefferson County,3065.0,18634.0
510,Wrightsville,31096,32.717320,-82.68926,New Hanover County,7046.0,20232.0
511,Yatesville,31097,32.912207,-84.14812,Lawrence County,1656.0,27000.0
512,Young Harris,30582,34.954638,-83.86484,Towns County,5038.0,15867.0


In [276]:
# #Extract values we need from the census data
# census_group = census_data.groupby(["City"]).agg({'Median Income':'mean', 'Population' : 'sum'})

# census = pd.DataFrame(census_group)
# census


,Median Income,Population
City,,
Abbeville,20880.0,4563.0
Acworth,33711.5,99683.0
Adairsville,24570.0,16138.0
Adel,25632.0,10836.0
Adrian,19562.0,3071.0
...,...,...
Wrens,18634.0,3065.0
Wrightsville,20232.0,7046.0
Yatesville,27000.0,1656.0


In [413]:
#merging hospital and census data
city_merge = pd.merge(hospital, census_data, how = 'outer', on = 'City')



city_merge = city_merge.rename(columns={"City": "Hospital City","Longtitude" : "Hospital Longitude", "County_x" : "Hospital County", "Latitude_x" : "Hospital Latitude", "ZIP" : "City Zip Code", "Zip Code" : "Hospital Zip Code", "Latitude_y" : "City Latiude", "Longitude" : "City Longitude", "County_y" : "City County"})



city_merge

,Hospital name,Hospital City,Hospital County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Hospital Latitude,Hospital Longitude,Street Address,State,Hospital Zip Code,Full Address,City Zip Code,City Latiude,City Longitude,City County,Population,Median Income
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.510205,-84.926304,1035 Red Bud Road Northeast,Georgia,30701.0,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA",30703.0,34.479066,-84.762161,Gordon County,40820.0,24110.0
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.779956,-84.783038,707 Old Dalton Ellijay Road,Georgia,30705.0,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070...",30705.0,34.753129,-84.775020,Los Angeles County,33460.0,24457.0
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.825169,-83.972128,915 Gordon Avenue,Georgia,31792.0,"915 Gordon Ave, Thomasville, GA 31792, USA",31758.0,30.868187,-83.927785,Thomas County,34781.0,24350.5
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795039,-83.559146,903 North Court Street,Georgia,31643.0,"903 N Court St, Quitman, GA 31643, USA",31643.0,30.783639,-83.553430,Wood County,9062.0,24644.0
4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.863815,-84.199066,1155 5th Street Southeast,Georgia,39828.0,"1155 5th St SE, Cairo, GA 39828, USA",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,NaN,Wrens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30833.0,33.200325,-82.376250,Jefferson County,3065.0,18634.0
549,NaN,Wrightsville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31096.0,32.717320,-82.689260,New Hanover County,7046.0,20232.0
550,NaN,Yatesville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31097.0,32.912207,-84.148120,Lawrence County,1656.0,27000.0
551,NaN,Young Harris,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30582.0,34.954638,-83.864840,Towns County,5038.0,15867.0


In [414]:
#extracting hospitals that did not have corresponding census data
city_merge['Joined'] = city_merge['Median Income'].isnull()
unjoined = city_merge.loc[city_merge['Joined'] == True]

unjoined = unjoined.loc[unjoined['State'] == 'Georgia']
unjoined

,Hospital name,Hospital City,Hospital County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Hospital Latitude,Hospital Longitude,...,State,Hospital Zip Code,Full Address,City Zip Code,City Latiude,City Longitude,City County,Population,Median Income,Joined
4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.863815,-84.199066,...,Georgia,39828.0,"1155 5th St SE, Cairo, GA 39828, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
38,Donalsonville Hospital,Donalsonville,Seminole County,65,1919,â,NaN,ChIJDROJ95Qz7YgR4AS_uCutkDI,31.050927,-84.880666,...,Georgia,39845.0,"102 Hospital Cir, Donalsonville, GA 39845, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
47,Emory Johns Creek Hospital,Johns Creek,Fulton County,118,2007,Emory,NaN,ChIJ8wevZy-Z9YgR5mpDmsMDM48,34.066497,-84.176237,...,Georgia,30097.0,"6325 Hospital Pkwy, Johns Creek, GA 30097, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
50,Fairview Park Hospital,Dublin,Laurens County,190,NaN,HCA,NaN,ChIJ00hnV80h8YgR-CDnNpspPns,32.532739,-82.950730,...,Georgia,31021.0,"200 Industrial Blvd, Dublin, GA 31021, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
56,Higgins General Hospital,Bremen,Haralson County,25,NaN,Tanner,NaN,ChIJYWQnyWPYiogRnm7ZzbYKhsI,33.716893,-85.144944,...,Georgia,30110.0,"200 Allen Memorial Dr, Bremen, GA 30110, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
64,LifeBrite Community Hospital of Early,Blakely,Early County,25,NaN,LifeBrite,Formerly Pioneer Community Hospital of Early,ChIJrc5daj-tkogRy9bc9VL33QQ,31.373318,-84.945705,...,Georgia,39823.0,"11740 Columbia Street, Blakely, GA 39823, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
67,Memorial Hospital and Manor,Bainbridge,Decatur County,80/107,1960,â,NaN,ChIJy4AyxxAZ7YgRUTmGeEmLfp4,30.902716,-84.552460,...,Georgia,39819.0,"1500 E Shotwell St, Bainbridge, GA 39819, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
69,Miller County Hospital,Colquitt,Miller County,25,1957,â,NaN,ChIJ03tgHp9H7YgRzI6aVejbskA,31.175429,-84.735244,...,Georgia,39837.0,"209 N Cuthbert St, Colquitt, GA 39837, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
88,Piedmont Athens Regional,Athens,Clarke County,350,1919,Piedmont,Formerly Athens Regional Medical Center,ChIJx6REsTht9ogRarSpPB_uhR4,33.961393,-83.397967,...,Georgia,30606.0,"1199 Prince Ave, Athens, GA 30606, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
101,Southeast Georgia Health System Camden,St. Marys,Camden County,40,NaN,SGHS,NaN,ChIJU6IsZ5cE5YgRaTwbgslwgUw,30.780916,-81.615147,...,Georgia,31558.0,"2000 Dan Proctor Dr, St Marys, GA 31558, USA",NaN,NaN,NaN,NaN,NaN,NaN,True


In [418]:
#Exporting necessary data to create mapping to update cities
unjoined_cities = unjoined['Hospital City'].unique()
city_map = pd.DataFrame(unjoined_cities)
city_map.to_csv(r"Resources/city_map.csv")

census_cities = census_data['City'].unique()
census_cities = pd.DataFrame(census_cities)
census_cities.to_csv(r"Resources/census_cities.csv")

In [420]:
#Importing the created mapping file in to fix discrepencies ('unjoined') outlined above
city_map = pd.read_csv("city_map_v2.csv")
del city_map['Unnamed: 0']

city_map

,city_map_old,city_map_new
0,Cairo,Cairo
1,Donalsonville,Donalsonville
2,Johns Creek,Alpharetta
3,Blakely,Blakely
4,Bainbridge,Bainbridge
5,Colquitt,Colquitt
6,St. Marys,Saint Marys
7,Cuthbert,Cuthbert
8,LaGrange,Lagrange
9,Peachtree Corners,Roswell


In [441]:
#Mapping in updated city names
updated_city_names = pd.merge(unjoined, city_map, how = 'inner', left_on = 'Hospital City', right_on = 'city_map_old')

#formatting
del updated_city_names['Hospital City']
del updated_city_names['city_map_old']
del updated_city_names['Median Income']
del updated_city_names['Population']
del updated_city_names['Joined']

#Using the mapped in columnn as the new value for city
updated_city_names = updated_city_names.rename(columns={"city_map_new" : "City"})


#Printing Results
updated_city_names

,Hospital name,Hospital County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Hospital Latitude,Hospital Longitude,Street Address,State,Hospital Zip Code,Full Address,City Zip Code,City Latiude,City Longitude,City County,City
0,Archbold - Grady General Hospital,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.863815,-84.199066,1155 5th Street Southeast,Georgia,39828.0,"1155 5th St SE, Cairo, GA 39828, USA",NaN,NaN,NaN,NaN,Cairo
1,Donalsonville Hospital,Seminole County,65,1919,â,NaN,ChIJDROJ95Qz7YgR4AS_uCutkDI,31.050927,-84.880666,102 Hospital Cir,Georgia,39845.0,"102 Hospital Cir, Donalsonville, GA 39845, USA",NaN,NaN,NaN,NaN,Donalsonville
2,Emory Johns Creek Hospital,Fulton County,118,2007,Emory,NaN,ChIJ8wevZy-Z9YgR5mpDmsMDM48,34.066497,-84.176237,6325 Hospital Parkway,Georgia,30097.0,"6325 Hospital Pkwy, Johns Creek, GA 30097, USA",NaN,NaN,NaN,NaN,Alpharetta
3,LifeBrite Community Hospital of Early,Early County,25,NaN,LifeBrite,Formerly Pioneer Community Hospital of Early,ChIJrc5daj-tkogRy9bc9VL33QQ,31.373318,-84.945705,11740 Columbia Street,Georgia,39823.0,"11740 Columbia Street, Blakely, GA 39823, USA",NaN,NaN,NaN,NaN,Blakely
4,Memorial Hospital and Manor,Decatur County,80/107,1960,â,NaN,ChIJy4AyxxAZ7YgRUTmGeEmLfp4,30.902716,-84.552460,1500 East Shotwell Street,Georgia,39819.0,"1500 E Shotwell St, Bainbridge, GA 39819, USA",NaN,NaN,NaN,NaN,Bainbridge
5,Miller County Hospital,Miller County,25,1957,â,NaN,ChIJ03tgHp9H7YgRzI6aVejbskA,31.175429,-84.735244,209 North Cuthbert Street,Georgia,39837.0,"209 N Cuthbert St, Colquitt, GA 39837, USA",NaN,NaN,NaN,NaN,Colquitt
6,Southeast Georgia Health System Camden,Camden County,40,NaN,SGHS,NaN,ChIJU6IsZ5cE5YgRaTwbgslwgUw,30.780916,-81.615147,2000 Dan Proctor Drive,Georgia,31558.0,"2000 Dan Proctor Dr, St Marys, GA 31558, USA",NaN,NaN,NaN,NaN,Saint Marys
7,Southwest Georgia Regional Medical Center,Randolph County,105,1916/1947,â,Formerly Patterson Hospital,ChIJwYwPs8Gw8ogRsx59PxkaP74,31.775420,-84.793018,361 Randolph Street,Georgia,39840.0,"361 Randolph St, Cuthbert, GA 39840, USA",NaN,NaN,NaN,NaN,Cuthbert
8,WellStar West Georgia Medical Center,Troup County,542,1937,WellStar,NaN,ChIJ40ZJ5vtWi4gRK6PnDb-BAPU,33.030940,-85.058637,1514 Vernon Road,Georgia,30240.0,"1514 Vernon Rd, LaGrange, GA 30240, USA",NaN,NaN,NaN,NaN,Lagrange
9,Eastside Medical Center,Gwinnett County,249,NaN,HCA,NaN,ChIJaTUBHwOh9YgRoHi7L2zoz4E,33.964011,-84.225939,5707 Peachtree Parkway,Georgia,30092.0,"5707 Peachtree Pkwy #100, Peachtree Corners, G...",NaN,NaN,NaN,NaN,Roswell


In [442]:
#Re-merging the data above with the census data
group2 = pd.merge(updated_city_names, census_data, how = 'inner', on = 'City', indicator = True)
joined2 = group2.loc[group2['_merge'] == 'both']
joined2

del joined2['City Longitude']
del joined2['City Lat']
del joined2['City County']


joined2 = joined2.rename(columns={"ZIP" : "City Zip Code", "Zip Code" : "Hospital Zip Code", "Latitude" : "City Latitude", "Longitude" : "City Longitude", "County" : "City County"})


joined2

,Hospital name,Hospital County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Hospital Latitude,Hospital Longitude,Street Address,...,City Zip Code,City Latiude,City,City Zip Code,City Latiude,City Longitude,City County,Population,Median Income,_merge
0,Emory Johns Creek Hospital,Fulton County,118,2007,Emory,NaN,ChIJ8wevZy-Z9YgR5mpDmsMDM48,34.066497,-84.176237,6325 Hospital Parkway,...,NaN,NaN,Alpharetta,30005,34.080035,-84.21929,Fulton County,189192.0,51690.25,both
1,Southeast Georgia Health System Camden,Camden County,40,NaN,SGHS,NaN,ChIJU6IsZ5cE5YgRaTwbgslwgUw,30.780916,-81.615147,2000 Dan Proctor Drive,...,NaN,NaN,Saint Marys,31558,30.761081,-81.56912,Jasper County,21307.0,29862.00,both
2,WellStar West Georgia Medical Center,Troup County,542,1937,WellStar,NaN,ChIJ40ZJ5vtWi4gRK6PnDb-BAPU,33.030940,-85.058637,1514 Vernon Road,...,NaN,NaN,Lagrange,30241,33.036715,-84.98895,Troup County,54380.0,23961.50,both
3,Eastside Medical Center,Gwinnett County,249,NaN,HCA,NaN,ChIJaTUBHwOh9YgRoHi7L2zoz4E,33.964011,-84.225939,5707 Peachtree Parkway,...,NaN,NaN,Roswell,30077,33.844371,-84.47405,Chaves County,99770.0,42613.00,both


In [433]:
#Looking if there's still unjoined records, will end up excluding this from the data
unjoined2 = group2.loc[group2['_merge'] == 'left_only']
unjoined2

,Hospital name,Hospital County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Hospital Latitude,Hospital Longitude,Street Address,...,City Longitude,City County,City,ZIP,Latitude,Longitude,County,Population,Median Income,_merge
0,Archbold - Grady General Hospital,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.863815,-84.199066,1155 5th Street Southeast,...,NaN,NaN,Cairo,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,Donalsonville Hospital,Seminole County,65,1919,â,NaN,ChIJDROJ95Qz7YgR4AS_uCutkDI,31.050927,-84.880666,102 Hospital Cir,...,NaN,NaN,Donalsonville,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,LifeBrite Community Hospital of Early,Early County,25,NaN,LifeBrite,Formerly Pioneer Community Hospital of Early,ChIJrc5daj-tkogRy9bc9VL33QQ,31.373318,-84.945705,11740 Columbia Street,...,NaN,NaN,Blakely,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,Memorial Hospital and Manor,Decatur County,80/107,1960,â,NaN,ChIJy4AyxxAZ7YgRUTmGeEmLfp4,30.902716,-84.552460,1500 East Shotwell Street,...,NaN,NaN,Bainbridge,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5,Miller County Hospital,Miller County,25,1957,â,NaN,ChIJ03tgHp9H7YgRzI6aVejbskA,31.175429,-84.735244,209 North Cuthbert Street,...,NaN,NaN,Colquitt,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7,Southwest Georgia Regional Medical Center,Randolph County,105,1916/1947,â,Formerly Patterson Hospital,ChIJwYwPs8Gw8ogRsx59PxkaP74,31.775420,-84.793018,361 Randolph Street,...,NaN,NaN,Cuthbert,NaN,NaN,NaN,NaN,NaN,NaN,left_only
10,WellStar Atlanta Medical Center South,Fulton County,183,NaN,WellStar,Formerly South Fulton Medical Center,ChIJA1sZb6UC9YgREjkaPkATQAU,33.680628,-84.426346,Cleveland Avenue River Park,...,NaN,NaN,East Point,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [428]:
#Extracting rows that did join successfully from the beginning between hospital and census data
group1 = city_merge.loc[city_merge['Joined'] == False]
group1


,Hospital name,Hospital City,Hospital County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Hospital Latitude,Hospital Longitude,...,State,Hospital Zip Code,Full Address,City Zip Code,City Latiude,City Longitude,City County,Population,Median Income,Joined
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.510205,-84.926304,...,Georgia,30701.0,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA",30703.0,34.479066,-84.762161,Gordon County,40820.0,24110.0,False
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.779956,-84.783038,...,Georgia,30705.0,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070...",30705.0,34.753129,-84.775020,Los Angeles County,33460.0,24457.0,False
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.825169,-83.972128,...,Georgia,31792.0,"915 Gordon Ave, Thomasville, GA 31792, USA",31758.0,30.868187,-83.927785,Thomas County,34781.0,24350.5,False
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795039,-83.559146,...,Georgia,31643.0,"903 N Court St, Quitman, GA 31643, USA",31643.0,30.783639,-83.553430,Wood County,9062.0,24644.0,False
5,Archbold - Mitchell County Hospital,Camilla,Mitchell County,25,NaN,Archbold,NaN,ChIJswuXyhic7YgRuk38qsPrYt4,31.229773,-84.205317,...,Georgia,31730.0,"90 E Stephens St, Camilla, GA 31730, USA",31730.0,31.209854,-84.236670,Mitchell County,8419.0,21584.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,NaN,Wrens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,30833.0,33.200325,-82.376250,Jefferson County,3065.0,18634.0,False
549,NaN,Wrightsville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,31096.0,32.717320,-82.689260,New Hanover County,7046.0,20232.0,False
550,NaN,Yatesville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,31097.0,32.912207,-84.148120,Lawrence County,1656.0,27000.0,False
551,NaN,Young Harris,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,30582.0,34.954638,-83.864840,Towns County,5038.0,15867.0,False


In [429]:
#Merging the two groups together
updated_data = pd.concat([group1,joined2])

del updated_data['Joined']
del updated_data['_merge']

updated_data

,Hospital name,Hospital City,Hospital County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Hospital Latitude,Hospital Longitude,...,State,Hospital Zip Code,Full Address,City Zip Code,City Latiude,City Longitude,City County,Population,Median Income,City
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.510205,-84.926304,...,Georgia,30701.0,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA",30703.0,34.479066,-84.762161,Gordon County,40820.0,24110.000000,NaN
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.779956,-84.783038,...,Georgia,30705.0,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070...",30705.0,34.753129,-84.775020,Los Angeles County,33460.0,24457.000000,NaN
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.825169,-83.972128,...,Georgia,31792.0,"915 Gordon Ave, Thomasville, GA 31792, USA",31758.0,30.868187,-83.927785,Thomas County,34781.0,24350.500000,NaN
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795039,-83.559146,...,Georgia,31643.0,"903 N Court St, Quitman, GA 31643, USA",31643.0,30.783639,-83.553430,Wood County,9062.0,24644.000000,NaN
5,Archbold - Mitchell County Hospital,Camilla,Mitchell County,25,NaN,Archbold,NaN,ChIJswuXyhic7YgRuk38qsPrYt4,31.229773,-84.205317,...,Georgia,31730.0,"90 E Stephens St, Camilla, GA 31730, USA",31730.0,31.209854,-84.236670,Mitchell County,8419.0,21584.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,NaN,Zebulon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,30295.0,33.101705,-84.312520,Wake County,4352.0,31654.000000,NaN
2,Emory Johns Creek Hospital,NaN,Fulton County,118,2007,Emory,NaN,ChIJ8wevZy-Z9YgR5mpDmsMDM48,34.066497,-84.176237,...,Georgia,30097.0,"6325 Hospital Pkwy, Johns Creek, GA 30097, USA",NaN,NaN,NaN,NaN,229248.0,53503.600000,Alpharetta
6,Southeast Georgia Health System Camden,NaN,Camden County,40,NaN,SGHS,NaN,ChIJU6IsZ5cE5YgRaTwbgslwgUw,30.780916,-81.615147,...,Georgia,31558.0,"2000 Dan Proctor Dr, St Marys, GA 31558, USA",NaN,NaN,NaN,NaN,21307.0,29862.000000,Saint Marys
8,WellStar West Georgia Medical Center,NaN,Troup County,542,1937,WellStar,NaN,ChIJ40ZJ5vtWi4gRK6PnDb-BAPU,33.030940,-85.058637,...,Georgia,30240.0,"1514 Vernon Rd, LaGrange, GA 30240, USA",NaN,NaN,NaN,NaN,79646.0,23825.333333,Lagrange


In [430]:
#Cleaning up data
updated_data['Acute Care Beds'] = updated_data['Acute Care Beds'].replace(to_replace='[citation needed]',value="",regex=True)

#updating datatypes
updated_data['Population'] = updated_data['Population'].astype(int)
updated_data['Median Income'] = updated_data['Median Income'].astype(int)
updated_data['Acute Care Beds'] = updated_data['Acute Care Beds'].astype(str)
updated_data['Acute Care Beds'] = updated_data['Acute Care Beds'].replace(to_replace='\[]',value="",regex=True)
updated_data['Acute Care Beds'] = updated_data['Acute Care Beds'].replace(to_replace='nan',value="0",regex=True)
updated_data['Acute Care Beds'] = updated_data['Acute Care Beds'].replace(to_replace='  ',value="",regex=True)
updated_data['Acute Care Beds'] = updated_data['Acute Care Beds'].astype(int)

updated_data


,Hospital name,Hospital City,Hospital County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Hospital Latitude,Hospital Longitude,...,State,Hospital Zip Code,Full Address,City Zip Code,City Latiude,City Longitude,City County,Population,Median Income,City
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.510205,-84.926304,...,Georgia,30701.0,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA",30703.0,34.479066,-84.762161,Gordon County,40820,24110,NaN
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.779956,-84.783038,...,Georgia,30705.0,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070...",30705.0,34.753129,-84.775020,Los Angeles County,33460,24457,NaN
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.825169,-83.972128,...,Georgia,31792.0,"915 Gordon Ave, Thomasville, GA 31792, USA",31758.0,30.868187,-83.927785,Thomas County,34781,24350,NaN
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795039,-83.559146,...,Georgia,31643.0,"903 N Court St, Quitman, GA 31643, USA",31643.0,30.783639,-83.553430,Wood County,9062,24644,NaN
5,Archbold - Mitchell County Hospital,Camilla,Mitchell County,25,NaN,Archbold,NaN,ChIJswuXyhic7YgRuk38qsPrYt4,31.229773,-84.205317,...,Georgia,31730.0,"90 E Stephens St, Camilla, GA 31730, USA",31730.0,31.209854,-84.236670,Mitchell County,8419,21584,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,NaN,Zebulon,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,30295.0,33.101705,-84.312520,Wake County,4352,31654,NaN
2,Emory Johns Creek Hospital,NaN,Fulton County,118,2007,Emory,NaN,ChIJ8wevZy-Z9YgR5mpDmsMDM48,34.066497,-84.176237,...,Georgia,30097.0,"6325 Hospital Pkwy, Johns Creek, GA 30097, USA",NaN,NaN,NaN,NaN,229248,53503,Alpharetta
6,Southeast Georgia Health System Camden,NaN,Camden County,40,NaN,SGHS,NaN,ChIJU6IsZ5cE5YgRaTwbgslwgUw,30.780916,-81.615147,...,Georgia,31558.0,"2000 Dan Proctor Dr, St Marys, GA 31558, USA",NaN,NaN,NaN,NaN,21307,29862,Saint Marys
8,WellStar West Georgia Medical Center,NaN,Troup County,542,1937,WellStar,NaN,ChIJ40ZJ5vtWi4gRK6PnDb-BAPU,33.030940,-85.058637,...,Georgia,30240.0,"1514 Vernon Rd, LaGrange, GA 30240, USA",NaN,NaN,NaN,NaN,79646,23825,Lagrange


In [ ]:
#Add code here to incorporate formatting Jerry did


In [431]:
#Exporting merged data
updated_data
updated_data.to_csv(r"Outputs/Finalized_Data.csv", index = False)

In [ ]:
#extracting just the hospital data
joined['Joined'] = joined['Place ID'].isnull()
hospitals = joined.loc[joined['Joined'] == False]

del hospitals['Unnamed: 0']
hospitals

In [319]:
#Plotting hospitals as the markers, city + 

hospitallist= hospitals["Hospital name"].tolist()


fig = gmaps.figure()

locations = hospitals[['Hospital Latitude', 'Hospital Longtitude']]


#Creating heat layer
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Name: {city}" for city in hospitallist])

fig.add_layer(markers)

fig

NameError: name 'hospitals' is not defined

In [ ]:
#heatmap of hospital beds per thousand people side by side with income and population

In [ ]:
#Summary statistics

hospital_group = hospital.groupby("City")

summary = round(hospital_group['Acute Care Beds'].agg(['mean','sum','median','var','std','sem']), 2)

summary